In [42]:
import glob, os, shutil
import pandas as pd
import numpy as np

In [43]:
pwd

'/Data/federated_learning/large_vlm_distillation_ood/Resnet18_classification/s_cars_ood'

In [5]:
os.mkdir('s_cars_ood')

In [6]:
os.mkdir('s_cars_ood/train/'),os.mkdir('s_cars_ood/test/')

(None, None)

In [3]:
ls

anno_test.csv                            s_cars_ood/
anno_train.csv                           stanfordCars_dataloader.py
car_data/                                train_resnet18_1.py
Classify_StanfordCars_on_resnet18.ipynb  train_resnet18.py
names.csv                                train_resnet18_StCars.py
play.py                                  Untitled.ipynb
plot_cifar_on_resnet18_train_results.py


In [9]:
# would like to do: 
"""
1. copy test to train data also same in csv
2. copy 10 directories back to test
3. strip each dataset (train, test) from directoies to sit in either train or test folders
"""

'\n1. copy test to train data also same in csv\n2. copy 10 directories back to test\n3. strip each dataset (train, test) from directoies to sit in either train or test folders\n'

In [44]:
root = '/Data/federated_learning/large_vlm_distillation_ood/Resnet18_classification/s_cars_ood/'

In [45]:
train_folds, test_folds = os.listdir(root+'train/'), os.listdir(root+'test/')

In [34]:
new_test_index = 8144
for fold in os.listdir(root+'test/'):
   for file in glob.glob(os.path.join(root+'test/'+fold, '*.jpg')):
       new_file_num = str(int(file.split('/')[-1].split('.')[0])+new_test_index)
       #print(root+'train/'+fold+'/'+new_file_num+'.jpg')
       shutil.copy(file, root+'train/'+fold+'/'+new_file_num+'.jpg')

In [46]:
df_train, df_test = pd.read_csv(root+'anno_train.csv',header=None), pd.read_csv(root+'anno_test.csv',header=None)

In [47]:
print(f'{df_test.head()} df_train shape {df_train.shape}, df_test shape {df_test.shape}')

           0    1    2    3    4    5
0  00001.jpg   30   52  246  147  181
1  00002.jpg  100   19  576  203  103
2  00003.jpg   51  105  968  659  145
3  00004.jpg   67   84  581  407  187
4  00005.jpg  140  151  593  339  185 df_train shape (8144, 6), df_test shape (8041, 6)


In [48]:
def add_value_to_filename(filename, value):
    # Extract numeric part and convert to integer
    num = int(filename.split('.')[0])
    # Add the desired value
    new_num = num + value
    # Format back to the original format with leading zeros
    new_filename = f'{new_num:05d}.jpg'
    return new_filename

In [52]:
os.mkdir('test/')

In [55]:
num_train_fold = len(os.listdir(root+'train/'))
from numpy.random import default_rng

rng = default_rng()
choices = rng.choice(num_train_fold, size=10, replace=False)
print(type(choices), choices)
selected_folds = [train_folds[i] for i in choices]

<class 'numpy.ndarray'> [171 161  94 187  76  31 116 113 141  34]


In [60]:
print(selected_folds)
for fold in selected_folds:
    shutil.move(root+'train/'+fold, root+'test/')
# at this point I moved 10 classes/folders back to test and the annotations should be prepared correctly as well

['Chevrolet Malibu Hybrid Sedan 2010', 'Volvo C30 Hatchback 2012', 'Audi 100 Sedan 1994', 'Ferrari California Convertible 2012', 'Audi S5 Coupe 2012', 'Dodge Caliber Wagon 2012', 'Ford F-150 Regular Cab 2007', 'Hyundai Veracruz SUV 2012', 'Audi TTS Coupe 2012', 'Buick Verano Sedan 2012']


In [61]:
df_test[0] = df_test[0].apply(add_value_to_filename, value=new_test_index)

In [66]:
print(df_train.shape)
print(df_train)
print(df_test.shape)
print(df_test)

(8144, 6)
              0    1    2     3     4    5
0     00001.jpg   39  116   569   375   14
1     00002.jpg   36  116   868   587    3
2     00003.jpg   85  109   601   381   91
3     00004.jpg  621  393  1484  1096  134
4     00005.jpg   14   36   133    99  106
...         ...  ...  ...   ...   ...  ...
8139  08140.jpg    3   44   423   336   78
8140  08141.jpg  138  150   706   523  196
8141  08142.jpg   26  246   660   449  163
8142  08143.jpg   78  526  1489   908  112
8143  08144.jpg   20  240   862   677   17

[8144 rows x 6 columns]
(8041, 6)
              0    1    2     3    4    5
0     08145.jpg   30   52   246  147  181
1     08146.jpg  100   19   576  203  103
2     08147.jpg   51  105   968  659  145
3     08148.jpg   67   84   581  407  187
4     08149.jpg  140  151   593  339  185
...         ...  ...  ...   ...  ...  ...
8036  16181.jpg   49   57  1169  669   63
8037  16182.jpg   23   18   640  459   16
8038  16183.jpg   33   27   602  252   17
8039  16184.jpg   3

In [67]:
df_labels = pd.concat([df_train, df_test], axis=0, ignore_index=True)

In [68]:
df_labels.to_csv('train_test_labels.csv', index=False)

In [79]:
len(os.listdir(root+'train/')), len(os.listdir(root+'test/'))
train_fold, test_fold = os.listdir(root+'train/'), os.listdir(root+'test/')

In [109]:
df_names = pd.read_csv('../names.csv',header=None)
matching_indices = df_names[df_names[0].isin(test_fold)].index.tolist()
print(matching_indices,type(matching_indices))
correct_ind_for_test = []
for ind in matching_indices:
    correct_ind_for_test.append(ind+1)
print(correct_ind_for_test)

[13, 16, 21, 48, 66, 82, 101, 113, 132, 192] <class 'list'>
[14, 17, 22, 49, 67, 83, 102, 114, 133, 193]


In [101]:
test_fold

['Dodge Caliber Wagon 2012',
 'Buick Verano Sedan 2012',
 'Audi S5 Coupe 2012',
 'Audi 100 Sedan 1994',
 'Hyundai Veracruz SUV 2012',
 'Ford F-150 Regular Cab 2007',
 'Audi TTS Coupe 2012',
 'Volvo C30 Hatchback 2012',
 'Chevrolet Malibu Hybrid Sedan 2010',
 'Ferrari California Convertible 2012']

In [112]:
#df_train.head(), df_test.head(), correct_ind_for_test, df_labels

In [114]:
print(df_labels.head())

           0    1    2     3     4    5
0  00001.jpg   39  116   569   375   14
1  00002.jpg   36  116   868   587    3
2  00003.jpg   85  109   601   381   91
3  00004.jpg  621  393  1484  1096  134
4  00005.jpg   14   36   133    99  106


In [133]:
mask = df_labels[5].isin(correct_ind_for_test)
new_df_test = df_labels[mask]
new_df_train = df_labels[~mask]
print(new_df_train.shape, new_df_test.shape)

(15366, 6) (819, 6)


In [1]:
new_df_test.to_csv('anno_test.csv')
new_df_train.to_csv('anno_train.csv')

NameError: name 'new_df_test' is not defined